In [1]:
# Install Dependencies
# This specific package of tf is needed to agree with keras-rl2.
!pip install tensorflow==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 44.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing i

In [2]:
!pip install keras-rl2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.1 MB/s eta 0:00:00


In [3]:
# Import Dependencies
import random
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Different agents within the Keras model
from rl.agents.dqn import DQNAgent
# policy based RL. Specific policy
from rl.policy import BoltzmannQPolicy
# For our agent we need to maintain memory
from rl.memory import SequentialMemory

Setting up RPSLS Environment

In [14]:
# Set up environment for agent to learn. Includes defining actions and rewards.
class RockPaperScissorsLizardSpockEnvironment:
     def __init__(self):
      self.actions = [0, 1, 2, 3, 4]  # rock: 0, paper: 1, scissors: 2, lizard: 3, spock: 4
      self.rewards = {(0, 0): 0, (0, 1): -1, (0, 2): 1, (0, 3): 1, (0, 4): -1,
       (1, 0): 1, (1, 1): 0, (1, 2): -1, (1, 3): -1, (1, 4): 1, (2, 0): -1,
        (2, 1): 1, (2, 2): 0, (2, 3): 1, (2, 4): -1, (3, 0): -1, (3, 1): 1,
         (3, 2): -1, (3, 3): 0, (3, 4): 1, (4, 0): 1, (4, 1): -1, (4, 2): 1, (4, 3): -1, (4, 4): 0
                      }

      self.max_steps = 10
      self.current_step = 0

     def step(self, action):
        opponent_action = self.select_opponent_action()
        reward = self.rewards[(action, opponent_action)]
        self.current_step += 1
        done = self.current_step >= self.max_steps
        info = {}
        return opponent_action, reward, done, info

     def reset(self):
        self.current_step = 0
        action = random.choice(self.actions)
        return action

     def select_opponent_action(self):
        # This strategy is just a simple example of adding probabilities to the choices.
        probabilities = [0.4, 0.2, 0.1, 0.1, 0.2]  # distribution from website data: rock: 0.4, paper: 0.2, scissors: 0.1, lizard: 0.1, spock: 0.2
        opponent_action = random.choices(self.actions, weights=probabilities)[0]
        return opponent_action


In [15]:
# Define the Q-network model 1
def build_model(states, actions):
    input_shape = (states,)  # Use the length of states for the input shape
    inputs = Input(shape=input_shape)
    x = Dense(24, activation='relu')(inputs)
    x = Dense(24, activation='relu')(x)
    outputs = Dense(actions, activation='linear')(x)  # Output layer with number of actions
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [16]:
# Set up the environment and agent
env = RockPaperScissorsLizardSpockEnvironment()
states = 1
actions = [0, 1, 2, 3, 4]
model = build_model(states, len(actions))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [17]:
# Summary of model
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 5)                 125       
                                                                 
Total params: 773
Trainable params: 773
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Set up DQN agent
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, memory=memory, policy=policy,
               nb_actions=len(actions), nb_steps_warmup=10, target_model_update=1e-2)
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])

In [19]:
# Train the DQN agent
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: 0.0553
1000 episodes - episode_reward: 0.553 [-9.000, 10.000] - loss: 0.678 - mae: 1.995 - mean_q: 2.619

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: 0.0768
1000 episodes - episode_reward: 0.768 [-8.000, 9.000] - loss: 0.891 - mae: 2.633 - mean_q: 3.450

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 87s 9ms/step - reward: 0.0748
1000 episodes - episode_reward: 0.748 [-8.000, 9.000] - loss: 0.904 - mae: 2.687 - mean_q: 3.500

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 88s 9ms/step - reward: 0.0838
1000 episodes - episode_reward: 0.838 [-8.000, 9.000] - loss: 0.927 - mae: 2.749 - mean_q: 3.578

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 88s 9ms/step - reward: 0.0671
done, took 430.851 sec

In [20]:
# Evaluate the trained agent
scores = dqn.test(env, nb_episodes=100, visualize=False)

Testing for 100 episodes ...
Episode 1: reward: 4.000, steps: 10
Episode 2: reward: 4.000, steps: 10
Episode 3: reward: 7.000, steps: 10
Episode 4: reward: 10.000, steps: 10
Episode 5: reward: 4.000, steps: 10
Episode 6: reward: 6.000, steps: 10
Episode 7: reward: -2.000, steps: 10
Episode 8: reward: 5.000, steps: 10
Episode 9: reward: 1.000, steps: 10
Episode 10: reward: 5.000, steps: 10
Episode 11: reward: 2.000, steps: 10
Episode 12: reward: 4.000, steps: 10
Episode 13: reward: 5.000, steps: 10
Episode 14: reward: 4.000, steps: 10
Episode 15: reward: 5.000, steps: 10
Episode 16: reward: 2.000, steps: 10
Episode 17: reward: 6.000, steps: 10
Episode 18: reward: 2.000, steps: 10
Episode 19: reward: 2.000, steps: 10
Episode 20: reward: 6.000, steps: 10
Episode 21: reward: 4.000, steps: 10
Episode 22: reward: 3.000, steps: 10
Episode 23: reward: 4.000, steps: 10
Episode 24: reward: 7.000, steps: 10
Episode 25: reward: 6.000, steps: 10
Episode 26: reward: 6.000, steps: 10
Episode 27: rewa

In [21]:
# Calculate sum, mean and median of the rewards
sum_reward = np.sum(scores.history['episode_reward'])
mean_reward = np.mean(scores.history['episode_reward'])
median_reward = np.median(scores.history['episode_reward'])

print("Sum Reward:", sum_reward)
print("Mean Reward:", mean_reward)
print("Median Reward:", median_reward)

Sum Reward: 418.0
Mean Reward: 4.18
Median Reward: 4.0
